In [1]:
import ds
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from matplotlib.colors import ListedColormap
import math

df = pd.read_csv("Cleaned_DF.csv")

In [2]:
df.drop("Unnamed: 0", axis = 1, inplace = True)

In [3]:
mask1 = (df['CryoSleep'] == 0)&(df['route']>3.5)&(df['Age']>12.5)
features = ['Age','VIP','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','Deck','Num','Side','route']
dz = df[mask1][features+["Transported"]]
dz.shape

(1921, 12)

In [4]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
#X = dz.drop("Transported",axis = 1)
y = dz["Transported"]
X = dz[features]
X_std = sc.fit_transform(X)
X_std

array([[ 0.36973726, -0.29625828, -0.54131821, ..., -0.83412407,
        -1.00888906,  1.13923301],
       [ 1.91308587,  3.37543307, -0.50077521, ..., -0.83412407,
         0.99118926,  1.13923301],
       [-0.11763599, -0.29625828, -0.54131821, ..., -0.83412407,
         0.99118926,  1.13923301],
       ...,
       [ 0.53219501,  3.37543307, -0.54131821, ..., -0.60168803,
        -1.00888906,  0.43418578],
       [-0.19886486, -0.29625828, -0.54131821, ...,  0.60792804,
         0.99118926,  0.43418578],
       [ 0.77588163, -0.29625828, -0.4225178 , ...,  0.60792804,
         0.99118926,  1.13923301]])

In [19]:
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
#lda = LDA(n_components = None)
#X_lda = lda.fit_transform(X_std,y)

In [22]:
np.set_printoptions(precision = 4)
mean_vecs = []
labels = [-1.0,1.0]
for label in range(len(labels)):
    mean_vecs.append(np.mean(X_std[y == labels[label]],axis = 0))
    print('MV %s: %s\n'%(label,mean_vecs[label-1]))

MV 0: [-0.0112  0.0021  0.1419 -0.2287 -0.1118  0.1372  0.123   0.1017  0.0451
 -0.0773 -0.1223]

MV 1: [-0.0112  0.0021  0.1419 -0.2287 -0.1118  0.1372  0.123   0.1017  0.0451
 -0.0773 -0.1223]



In [ ]:
d = 

In [ ]:
X_lda = X_std.dot(w)

In [14]:
colors = ['r','b']
markers = ['s','x']
for l,c,m in zip(np.unique(y),colors, markers):
    plt.scatter(X_lda[y == l, 0],X_lda[y == l, 1],c = c, label = l, marker = m)

plt.xlabel('LD 1')
plt.ylabel('LD 2')
plt.legend(loc = 'lower left')
plt.tight_layout()
plt.show

IndexError: index 1 is out of bounds for axis 1 with size 1

In [10]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver = 'lbfgs')
ppn_test = lr.fit(X_lda,y)
ds.plot_decision_regions(X_lda,y,classifier = ppn_test,resolution = 0.1)

IndexError: index 1 is out of bounds for axis 1 with size 1

In [ ]:
lda_df = pd.DataFrame(X_lda,index = dz.index,columns = ['par1','par2'])
lda_df["Transported"] = y

lda_df.head(20)